In [1]:
import csv
import numpy as np
import sys
import pandas as pd
import itertools
import math
import time
import os

from sklearn import svm, linear_model, neighbors
from sklearn import tree, ensemble
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB

import networkx as nx

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
import numbers
def multimetric_score(estimator, X_test, y_test, scorers):
    """Return a dict of score for multimetric scoring"""
    scores = {}
    for name, scorer in scorers.items():
        if y_test is None:
            score = scorer(estimator, X_test)
        else:
            score = scorer(estimator, X_test, y_test)

        if hasattr(score, 'item'):
            try:
                # e.g. unwrap memmapped scalars
                score = score.item()
            except ValueError:
                # non-scalar?
                pass
        scores[name] = score

        if not isinstance(score, numbers.Number):
            raise ValueError("scoring must return a number, got %s (%s) "
                             "instead. (scorer=%s)"
                             % (str(score), type(score), name))
    return scores

In [3]:
def addLinkToNetwork(net,source, target):
    if source in net:
        if target not in net[source]:  
            net[source].append(target)
    else:
        net[source] =[target]

def removeLinkToNetwork(net, source, target):
    if source in net:
        if target in net[source]:
            net[source].remove(target)

In [4]:
def getNeighbors(net, x):
    if x in net:
        return net[x]
    else:
        return []
    
def nStepNeighbors(net, x, max_length):
    Found = dict()
    Found[x] = True
    NewSearch = []
    NewSearch.append(x)
    for curDeg in range(max_length):
        #print "Iteration ----------",curDeg
        OldSearch=[]
        while len(NewSearch) != 0:		
            OldSearch.append( NewSearch.pop())
        
        while len(OldSearch) != 0:
            vi = OldSearch.pop()
            
            for vj in getLinks(net,vi):
                if vj not in Found:
                    Found[vj]=True
                    NewSearch.append(vj)
    
    return Found.keys()
    
def getCommonNeighborsOfNeighbors(net, x):
    neighbors = list() 
    if x in net:
        for c in getNeighbors_(net, x):
            neighbors.append( set(getNeighbors_(net, c)))
    
        if len (neighbors) == 0: return []
        if len (neighbors) == 1: return neighbors[0]
        common =set.intersection(*neighbors)
        return common
    return []

def commonNeighbors(net, x, y, weights={}):
    if x== y:
        return len(getNeighbors(net,x))

    commons =set(getNeighbors(net,x)).intersection(getNeighbors(net,y))
    return len(commons)

def jaccard(net,x,y,weights={}):
    if x == y:
        return 1

    commons =set(getNeighbors(net,x)).intersection(getNeighbors(net,y))
    unions = set(getNeighbors(net,x)).union(getNeighbors(net,y))
    if len(unions) == 0:
        return 0.0
    return float(len(commons))/len(unions)
def adamic(net,x,y,weights={}):
    commons =set(getNeighbors(net,x)).intersection(getNeighbors(net,y))
    score =0.0
    for n in commons:
        score += 1.0/math.log(float(len(getNeighbors(net,n))))

    return score

def propFlowNet(net, start, end, weights={}):
    return 1
    global  allRanks
    if start not in allRanks:
        allRanks[start]=propFlow(net,start,weights)
    #print start,len(allRanks[start])
    #print net[start]
    if start in allRanks:
        if end in allRanks[start]:
            return allRanks[start][end]
        else:
            return 0.0
    return allRanks[start][end]

def propFlow(net, vs, weights={},  max_length=3, decayFactor= 0.15, topN=10, itemtype="mo" ):

    Found=dict()
    NewSearch=[]
    S=dict()
    Found[vs] =True
    NewSearch.append(vs)
    S[vs]=1.0
    #print "start node",vs
    #print net[vs]
    for curDeg in range(max_length):
        #print "Iteration ----------",curDeg
        OldSearch=[]
        while len(NewSearch) != 0:		
            OldSearch.append( NewSearch.pop())
        while len(OldSearch) != 0:
            vi =OldSearch.pop()
            NodeInput =S[vi]
            SumOutput =0
            for vj in getNeighbors(net,vi):
                if (vi,vj) in weights:
                    weight =weights[(vi,vj)]
                else:
                    weight=1.0
                SumOutput +=weight
            Flow =0
            if SumOutput ==0:
                SumOutput=1.0

            for vj in getNeighbors(net,vi):
                if (vi,vj) in weights:
                    weight =weights[(vi,vj)]
                else:
                    weight=1.0

                Flow = NodeInput * weight*(1-decayFactor)/SumOutput
                if vj in S:
                    S[vj] += Flow
                else:
                    S[vj] =Flow

                #print (vj, Flow)
                #print net[vj]
                if vj not in Found:
                    Found[vj]=True
                    NewSearch.append(vj)
    #print S
    return S	

def personalizedPagerank(net, start=None, damping_factor=0.85,max_iter=100,min_delta=0.00001, weights={}):

    nodes= net.keys()
    numNodes = len(nodes)
    if numNodes == 0:
        return {}
    min_value= (1.0-damping_factor)/numNodes
    pagerank = dict.fromkeys(nodes,1.0/numNodes)
    for i in range(max_iter):
        diff =0
        for node in nodes:
            rank = 0.0
            for neigh in net[node]:
                if (node,neigh) in weights:
                    weight =weights[(node,neigh)]
                else:
                    weight=1.0
                rank += damping_factor*weight*pagerank[neigh]/len(net[neigh])

            if type(start)==list:
                if  node in start:
                    rank +=min_value	
            else:
                if start == node:
                    rank +=min_value

            diff += abs(pagerank[node]-rank)
            pagerank[node]=rank
        if diff < min_delta:
            break

    return pagerank


In [5]:
goldindfile = "../data/input/unified-gold-standard-umls.txt"
embdfilename = "vectors/Entity2Vec_sg_200_5_5_15_2_500_d5_uniform.txt"
emb_df = pd.read_csv(embdfilename, delimiter='\t') 

gold_df= pd.read_csv(goldindfile, delimiter='\t')

In [6]:
emb_df.head()

,Entity,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,...,feature190,feature191,feature192,feature193,feature194,feature195,feature196,feature197,feature198,feature199
0,DB02133,-0.051658,0.163778,-0.361194,-0.087759,0.297820,0.305669,0.236298,0.254304,0.135926,...,-0.507540,-0.126325,0.208918,0.078276,-0.395061,-0.024653,0.245934,-0.024440,0.300537,0.041316
1,Fingerprint10,0.098235,0.485436,-0.309485,-0.256898,0.789238,0.328992,0.827521,-0.150584,-0.397627,...,-0.523152,-0.109714,0.122435,-0.000810,-0.030355,0.136024,0.280323,-0.314951,0.541641,0.045266
2,DB02863,-0.015989,0.141269,-0.162110,0.091442,0.441140,0.293197,0.156931,0.195844,0.298615,...,-0.407804,-0.057749,0.091881,-0.127052,-0.465435,-0.146370,0.241991,0.223220,0.206277,-0.025753
3,DB00258,0.203198,0.468460,-0.333941,0.534330,0.675704,-0.003532,0.554302,-0.210865,-0.257030,...,-0.178686,-0.310188,0.208252,-0.585612,-0.160281,-0.080312,0.326872,0.102249,-0.090835,-0.027320
4,DB08913,0.257988,0.621600,-0.641045,0.271267,0.463862,-0.259994,0.584031,0.013617,-0.112262,...,-0.528301,0.071888,-0.333639,-0.108896,0.123948,-0.096030,0.363442,0.126367,0.199852,-0.012021


In [7]:
bio_entities = emb_df.Entity.unique()
bio_entities=list(bio_entities)

drugDiseaseKnown = set([tuple(x) for x in  gold_df[['Drug','Disease']].values])

positives = gold_df[['Drug','Disease']]
positives = positives[positives.Drug.isin(bio_entities) & positives.Disease.isin(bio_entities)]
positives['Class'] = 1 

In [8]:
drugDiseaseKnown = set([tuple(x) for x in  positives[['Drug','Disease']].values])
pairs=[]
classes=[]
drugs = set(positives['Drug'].unique())
diseases = set(positives['Disease'].unique())
print ("commonDiseases",len(diseases))
print ("commonDrugs",len(drugs))
for dr in drugs:
    for di in diseases:
        if (dr,di)  in drugDiseaseKnown:
            cls=1
        else:
            cls=0
        pairs.append((dr,di))
        classes.append(cls)

pairs = np.array(pairs)        
classes = np.array(classes)

indices = np.where(classes == 1)
positives = pd.DataFrame(list(zip(pairs[indices][:,0],pairs[indices][:,1],classes[indices])), columns=['Drug','Disease','Class'])

indices = np.where(classes == 0)
all_negatives = pd.DataFrame(list(zip(pairs[indices][:,0],pairs[indices][:,1],classes[indices])), columns=['Drug','Disease','Class'])

print("INDI size: ", len(positives))
print("non-INDI size: ",len(all_negatives))

negatives = all_negatives.sample(2*len(positives)) # for balanced class

pair_df = pd.concat([positives,negatives], ignore_index=True)
print("Train size: ", len(pair_df))

commonDiseases 1377
commonDrugs 1453
INDI size:  7846
non-INDI size:  1992935
Train size:  23538


In [9]:
def netFeatureSet(net, n, n2, cls):
    PF = propFlowNet(net,n, n2)
    JC = jaccard(net,n,n2)
    AA = adamic(net,n,n2)
    CN = commonNeighbors(net,n,n2)
    print (PF, JC, AA, CN)
    return n, n2, cls, PF, JC, AA, CN

def getNumberOfNeighors(G, x):
    count = 0
    for y in G[x]:
        count += len(G[y])
    return count

def adamic(net,x,y,weights={}):
    commons =set(getNeighbors(net,x)).intersection(getNeighbors(net,y))
    score =0.0
    for n in commons:
        score += 1.0/math.log(float(len(getNeighbors(net,n))))

    return score

def getNumNeighs(G, x):
    if x in G:
        return len(G[x])
    return 0

def prefenAttach(G, n, n2):
    if n in G and n2 in G:
        return len(G[n])*len(G[n2])
    return 0

def PF_sim(G, x, y, weights ={}):
    addLater = False
    if G.has_edge(x,y):
        G.remove_edge(x,y)
        addLater = True
    PF = propFlow(G,x,max_length=5, weights=weights)
    if addLater:
        G.add_edge(x,y)
    
    if y in PF:
        return PF[y]
    else:
        return 0

def networkXFeatureSet(G, source, target, cls):
    if source in G.nodes and target in G.node:
        paths = nx.all_simple_paths(G, source, target, cutoff=3) # paths with length 3 between s and t 
        num_paths = len(list(paths))
    else:
        num_paths = 0
     
    if source in G.nodes:
        all_paths_3 = getNumberOfNeighors(G, source)
    else:
        all_paths_3 = 1 
    total_neigh_st= getNumNeighs(G,source)+ getNumNeighs(G,target) 
    
    if total_neigh_st <= 1:
        total_neigh_st += 0.5
        
    CN = num_paths
    JC = num_paths/all_paths_3
    PF = PF_sim(G, source, target) 
    #PA = prefenAttach(G, source, target)
    AA = num_paths/math.log(float(total_neigh_st))
    #print (PF, JC, AA, CN, cls)
    return source, target, cls, PF, JC, AA, CN

def linkPredSpark(bc_graph, pairs, classes):
    global  allRanks
    allRanks ={}

    pairList = list(zip(pairs[:,0],pairs[:,1],classes))

    start_time =time.time()
    rdd = sc.parallelize(pairList).map(lambda x: networkXFeatureSet(bc_graph.value, x[0], x[1], x[2] ))
    pair_df = rdd.collect()
    elapsed_time = time.time() - start_time
    #print ('Time elapsed to generate walks:',time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
    return pd.DataFrame(pair_df, columns=['Drug','Disease','Class', 'PF','JC','AA','CN'])

In [10]:
import findspark
findspark.init()

from pyspark import SparkConf, SparkContext

In [11]:
config = SparkConf()
config.setMaster("local[6]")
#config.set("spark.executor.memory", "2g")
sc = SparkContext(conf=config)
print (sc)

<SparkContext master=local[6] appName=pyspark-shell>


In [17]:
def get_scores(clf, X_new, y_new):

    scoring = ['precision', 'recall', 'accuracy', 'roc_auc', 'f1', 'average_precision']
    scorers, multimetric = metrics.scorer._check_multimetric_scoring(clf, scoring=scoring)
    #print(scorers)
    scores = multimetric_score(clf, X_new, y_new, scorers)
    return scores


def crossvalid(train_df, test_df): 
    features_cols= train_df.columns.difference(['Drug','Disease' ,'Class', 'Entity_x', 'Entity_y'])
    X=train_df[features_cols].values
    y=train_df['Class'].values.ravel()

    X_new=test_df[features_cols].values
    y_new=test_df['Class'].values.ravel()

    nb_model = GaussianNB()
    nb_model.fit(X,y)
    nb_scores = get_scores(nb_model, X_new, y_new)
    
    logistic_model = linear_model.LogisticRegression()
    logistic_model.fit(X,y)
    lr_scores = get_scores(logistic_model, X_new, y_new)
    
    rf_model = ensemble.RandomForestClassifier(n_estimators=200, n_jobs=-1)
    rf_model.fit(X,y)
    rf_scores = get_scores(rf_model, X_new, y_new)
    
    #clf = ensemble.RandomForestClassifier(n_estimators=100
    return nb_scores,lr_scores, rf_scores

In [13]:
def runa10foldcv(pair_df, emb_df,  cv):
    nb_scores_df = pd.DataFrame()
    lr_scores_df = pd.DataFrame()
    rf_scores_df = pd.DataFrame()
    
    for i, (train_idx, test_idx) in enumerate(cv):

        train = pair_df.loc[train_idx]
        test = pair_df.loc[test_idx]
        G = nx.Graph()
        for index, row in train[train.Class == 1 ].iterrows():
            drug=row['Drug']
            disease=row['Disease']
            G.add_edge(drug, disease)


        bc_net=sc.broadcast(G)
        #print (len(bc_net.value))

        test_df = linkPredSpark(bc_net, test[['Drug','Disease']].values, test.Class)
        train_df = linkPredSpark(bc_net, train[['Drug','Disease']].values, train.Class)

        train_df = train_df.merge(emb_df, left_on='Drug', right_on='Entity').merge(emb_df, left_on='Disease', right_on='Entity')
        test_df =  test_df.merge(emb_df, left_on='Drug', right_on='Entity').merge(emb_df, left_on='Disease', right_on='Entity')

        nb_scores,lr_scores, rf_scores = crossvalid(train_df, test_df)

        nb_scores_df = nb_scores_df.append(nb_scores, ignore_index=True)
        lr_scores_df = lr_scores_df.append(lr_scores, ignore_index=True)
        rf_scores_df = rf_scores_df.append(rf_scores, ignore_index=True)
    return nb_scores_df,lr_scores_df, rf_scores_df

In [18]:
from sklearn.model_selection import StratifiedKFold

#skf = StratifiedShuffleSplit(n_splits=10)

from sklearn.model_selection import StratifiedKFold

n_run = 10
nb_scores_df = pd.DataFrame()
lr_scores_df = pd.DataFrame()
rf_scores_df = pd.DataFrame()
for k in range(n_run):
    print ('run',k)
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=k)
    cv = skf.split(pair_df, pair_df.Class)
    nb_scores,lr_scores, rf_scores = runa10foldcv(pair_df,emb_df,  cv)
    nb_scores_df = nb_scores_df.append(nb_scores.mean(), ignore_index=True)
    lr_scores_df = lr_scores_df.append(lr_scores.mean(), ignore_index=True)
    rf_scores_df = rf_scores_df.append(rf_scores.mean(), ignore_index=True)

fold 0
fold 1
fold 2
fold 3
fold 4
fold 5
fold 6
fold 7
fold 8
fold 9


In [20]:
outfolder='results/'
if not os.path.isdir(outfolder):
    os.mkdir(outfolder)

nb_scores_df.to_csv(os.path.join(outfolder,'GEPDI_NETWORK_RDF2Vec_nb_results_pred_CV.csv'))
lr_scores_df.to_csv(os.path.join(outfolder,'GEPDI_NETWORK_RDF2Vec_lr_results_pred_CV.csv'))
rf_scores_df.to_csv(os.path.join(outfolder,'GEPDI_NETWORK_RDF2Vec_rf_results_pred_CV.csv'))

In [21]:
all_scores_df = pd.DataFrame()
all_scores_df = all_scores_df.append(lr_scores_df.mean(), ignore_index=True)
all_scores_df = all_scores_df.append(nb_scores_df.mean(), ignore_index=True)
all_scores_df = all_scores_df.append(rf_scores_df.mean(), ignore_index=True)

In [22]:
all_scores_df[['accuracy','roc_auc','average_precision','precision','recall','f1']]

,accuracy,roc_auc,average_precision,precision,recall,f1
0,0.891614,0.912680,0.896054,0.927522,0.732132,0.818204
1,0.882105,0.866904,0.848688,0.929408,0.699516,0.798114
2,0.891155,0.925510,0.907602,0.862638,0.801263,0.830704
